In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26240381
paper_name = 'tigano_ottonello_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/nar-03484-z-2014-File007.xlsx', sheet_name='Suppl_TabS1', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 489 x 10


In [10]:
original_data.head()

,Systematic name,Gene,Description,Chr,Human ortholog,ORF Type,GO Biological Process category (P-value),"Ethanol, 30°C","Ethanol, 37°C",Unnamed: 9
0,YAL002W,VPS8,Membrane-binding component of the CORVET compl...,1,YES,Verified,late endosome to vacuole transport [GO:0045324...,0,HS,NEW
1,YAL013W,DEP1,Component of the Rpd3L histone deacetylase com...,1,NaN,Verified,regulation of transcription from RNA polymeras...,MS,HS,NaN
2,YAL021C,CCR4,Component of the CCR4-NOT transcriptional comp...,1,YES,Verified,regulation of transcription from RNA polymeras...,LS,LS,NaN
3,YAL039C,CYC3,Cytochrome c heme lyase (holocytochrome c synt...,1,YES,Verified,cytochrome c-heme linkage [GO:0018063] (P-valu...,HS,HS,NaN
4,YAL044C,GCV3,H subunit of the mitochondrial glycine decarbo...,1,YES,Verified,glycine decarboxylation via glycine cleavage s...,MS,HS,NaN


In [11]:
original_data['orf'] = original_data['Systematic name'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Systematic name, Gene, Description, Chr, Human ortholog, ORF Type, GO Biological Process category (P-value), Ethanol, 30°C, Ethanol, 37°C, Unnamed: 9, orf]
Index: []


In [15]:
original_data.set_index('orf', inplace=True)

In [17]:
data_switch = {'HS': -3, 'MS': -2, 'LS': -1}
data_cols = ['Ethanol, 30°C','Ethanol, 37°C']

for c in data_cols:
    original_data[c] = original_data[c].apply(lambda x: data_switch[x] if x in data_switch.keys() else x)

In [18]:
original_data = original_data[data_cols].copy()

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(489, 2)

# Load & process tested strains

In [22]:
tested = pd.read_excel('raw_data/Tigano et al NAR 2015_List of tested yeast mutant strains.xlsx', sheet_name='Foglio1')

In [23]:
tested.head()

,ORF name,strain,batch,plate,row,column
0,YAL002W,BY4739,01_1,101,E,1
1,YAL004W,BY4739,01_1,101,D,11
2,YAL005C,BY4739,01_1,101,D,12
3,YAL007C,BY4739,01_1,101,D,10
4,YAL008W,BY4739,01_1,101,D,9


In [24]:
tested['orf'] = tested['ORF name'].astype(str)
tested['orf'] = clean_orf(tested['orf'])

In [25]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF name, strain, batch, plate, row, column, orf]
Index: []


In [27]:
tested_orfs = np.unique(tested['orf'].values)

In [28]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [695, 694]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,695,694
data_type,value,value
orf,,
YAL002W,0,-3
YAL004W,0,0
YAL005C,0,0
YAL007C,0,0
YAL008W,0,0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,695,694
,data_type,value,value
gene_id,orf,,
2,YAL002W,0,-3
1863,YAL004W,0,0
4,YAL005C,0,0
5,YAL007C,0,0
6,YAL008W,0,0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,695,694,695,694
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0,-3,0.0,-3.479389
1863,YAL004W,0,0,0.0,0.000000
4,YAL005C,0,0,0.0,0.000000
5,YAL007C,0,0,0.0,0.000000
6,YAL008W,0,0,0.0,0.000000


# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 26240381...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.97s/it]

Updating the data_modified_on field...
